# Generating a list of unique latitudes and longitudes for iteration

#### Import dependencies

In [1]:
import pandas as pd

#### Read in excel file

In [2]:
xlsx = pd.read_excel('lat_lon_grouped.xlsx')

#### Create dataframe and view head

In [3]:
df = pd.DataFrame(xlsx)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,lat_group,long_group
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,0,1955,0,98178,47.5112,-122.257,1340,5650,47.516708,-122.253
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,47.716540,-122.309
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,0,1933,0,98028,47.7379,-122.233,2720,8062,47.738744,-122.225
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,910,1965,0,98136,47.5208,-122.393,1360,5000,47.516708,-122.393
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,0,1987,0,98074,47.6168,-122.045,1800,7503,47.616624,-122.057


#### How many unique lat,long groups are there?

- empty list is created to hold unique latlong pairs
- dataframe is iterated over, each time:
    - concatenating latitude and longitude to create a unique latlong pair
    - testing to see if that pair exists in the list
        - if not, it is appended to the list

In [4]:
grouplist = []
for index, row in df.iterrows():
    coordgroup = str(row['lat_group']) + ',' + str(row['long_group'])
    if coordgroup in grouplist:
        pass
    else:
        grouplist.append(coordgroup)

#### Length of list is checked

In [5]:
len(grouplist)

855

So, looks like there are 855 unique latlong pairs
- each of these pairs can be fed to an api to get location information for the area
- however, since I'm not sure how to join that resulting dataset back to the original (on two columns at once), I think it is a good idea to create a 'latlong' column with the unique latlong pairs to join future datasets on.

To do this, we recycle a bit of the code above and add to it.

In [7]:
df['lat+long_group'] = ''

for index, row in df.iterrows():
    coordgroup = str(row['lat_group']) + ',' + str(row['long_group'])
    df.set_value(index,'lat+long_group',coordgroup)
    
df.head()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,lat_group,long_group,lat+long_group
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,1955,0,98178,47.5112,-122.257,1340,5650,47.516708,-122.253,"47.51670803571437,-122.2529999999999"
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,1951,1991,98125,47.7210,-122.319,1690,7639,47.716540,-122.309,"47.71654017857156,-122.3089999999999"
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,1933,0,98028,47.7379,-122.233,2720,8062,47.738744,-122.225,"47.73874375000014,-122.2249999999999"
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,1965,0,98136,47.5208,-122.393,1360,5000,47.516708,-122.393,"47.51670803571437,-122.393"
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,1987,0,98074,47.6168,-122.045,1800,7503,47.616624,-122.057,"47.61662410714297,-122.0569999999999"


Great, now we have a unique identifier to join future datasets on.

Next, we can create a new dataframe, with one row for every unique latlong pair
- This can be iterated over to feed into an api, with the resulting data put into a new column
- Then this data can be joined to the above dataset

In [8]:
df2 = df.iloc[:,21:24]
df2.head()

,lat_group,long_group,lat+long_group
0,47.516708,-122.253,"47.51670803571437,-122.2529999999999"
1,47.716540,-122.309,"47.71654017857156,-122.3089999999999"
2,47.738744,-122.225,"47.73874375000014,-122.2249999999999"
3,47.516708,-122.393,"47.51670803571437,-122.393"
4,47.616624,-122.057,"47.61662410714297,-122.0569999999999"


In [9]:
len(df2)

21436

In [10]:
df3 = df2.drop_duplicates()

In [11]:
len(df3)

855

In [12]:
df3.head()

,lat_group,long_group,lat+long_group
0,47.516708,-122.253,"47.51670803571437,-122.2529999999999"
1,47.716540,-122.309,"47.71654017857156,-122.3089999999999"
2,47.738744,-122.225,"47.73874375000014,-122.2249999999999"
3,47.516708,-122.393,"47.51670803571437,-122.393"
4,47.616624,-122.057,"47.61662410714297,-122.0569999999999"


So now we have df3, a dataframe with 3 columns:
- lat_group, for feeding into api
- long_group, for feeding into api
- lat+long_group, for joining with original dataset

It has no duplicates, making for efficient api calling.
- it contains 855 unique locations

In [15]:
df.to_csv('joinable_kc_dataset.csv', index=False)
df3.to_csv('unique_latlongs.csv', index=False)